<a href="https://colab.research.google.com/github/kanri3/deep_learning_day3_day4/blob/main/pytorch_hands_on_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### リスト1-1　ニューロンのモデル設計と活性化関数

- ニューロンへの入力＝$(w_1 \times X_1)+(w_2 \times X_2)+b$
- ニューロンからの出力＝$a((w_1 \times X_1)+(w_2 \times X_2)+b)$
  - $a()$は活性化関数を意味する。つまりニューロンの入力結果を、活性化関数で変換したうえで、出力する
  - 今回の活性化関数は、**tanh**関数とする
- ニューロンの構造とデータ入力：座標$(X_1, X_2)$
  - 入力の数（`INPUT_FEATURES`）は、$X_1$と$X_2$で**2つ**
  - ニューロンの数（`OUTPUT_NEURONS`）は、**1つ**

In [1]:
import torch       # ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn  # 「ニューラルネットワーク」モジュールの別名定義

# 定数（モデル定義時に必要となるもの）
INPUT_FEATURES = 2  # 入力（特徴）の数： 2
OUTPUT_NEURONS = 1  # ニューロンの数： 1

# 変数（モデル定義時に必要となるもの）
activation = torch.nn.Tanh()  # 活性化関数： tanh関数

# 「torch.nn.Moduleクラスのサブクラス化」によるモデルの定義
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 層（layer：レイヤー）を定義
        self.layer1 = nn.Linear(  # Linearは「全結合層」を指す
            INPUT_FEATURES,       # データ（特徴）の入力ユニット数
            OUTPUT_NEURONS)       # 出力結果への出力ユニット数

    def forward(self, input):
        # フォワードパスを定義
        output = activation(self.layer1(input))  # 活性化関数は変数として定義
        # 「出力＝活性化関数（第n層（入力））」の形式で記述する。
        # 層（layer）を重ねる場合は、同様の記述を続ければよい（第3回＝後述）。
        # 「出力（output）」は次の層（layer）への「入力（input）」に使う。
        # 慣例では入力も出力も「x」と同じ変数名で記述する（よって以下では「x」と書く）
        return output

# モデル（NeuralNetworkクラス）のインスタンス化
model = NeuralNetwork()
model   # モデルの内容を出力

NeuralNetwork(
  (layer1): Linear(in_features=2, out_features=1, bias=True)
)

このコードのポイント：
- `torch.nn.Module`クラスを継承して独自にモデル用クラスを定義する。Pythonの「モジュール」と紛らわしいので、本稿では「`torch.nn.Module`」と表記する
    - `__init__`関数にレイヤー（層）を定義する
    - `forward`関数にフォワードパス（＝活性化関数で変換しながらデータを流す処理）を実装する
    - ちなみにバックプロパゲーション（誤差逆伝播）のための`backward`関数は自動微分機能により自動作成される（後述）

# 第2回　PyTorchのテンソル＆データ型のチートシート

## ■（4）PyTorchの基礎： テンソルとデータ型

### 表4-1　PyTorchのデータ型

先ほどのリスト1-3では、`torch.tensor([0.8])`というコードでPyTorchのテンソル（`torch.Tensor`値）を作成した。PyTorchでデータや数値を扱うには、このテンソル形式にする必要がある。  
この例で分かるように、Pythonの数値やリスト値を`torch.Tensor`値に変換するのは難しくない。  
ここでは、テンソルを作成／変換する基本的なコードをチートシート的に書き出しておく。  
よく分からないところがあれば、下記の公式チュートリアルの説明を参照してほしい（※Chromeの［日本語に翻訳］機能を使えば、日本語で読める）。
- 公式チュートリアル： [What is PyTorch? — PyTorch Tutorials 1.4.0 documentation](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py)
- APIドキュメント： [torch — PyTorch master documentation](https://pytorch.org/docs/stable/torch.html#tensors)

また、テンソルの中に含める数値には、PyTorch独自のデータ型（`torch.dtypes`）がある。ただし統一データ型（ある1つのテンソル内に含まれる全要素の数値は全て同じデータ型）となっており、例えば`torch.float`の場合は、全ての要素の数値が「32-bitの浮動小数点」として扱われるので注意してほしい。基本的に`torch.float`か`torch.int`しか使わない。

データ型 | dtype属性への記述 | 対応するPython／NumPy（np）のデータ型
---------|-----------------|--------------------------------
Boolean（真偽値）|torch.bool|bool／np.bool
8-bitの符号なし整数|torch.uint8|int／np.uint8
8-bitの符号付き整数|torch.int8|int／np.int8
16-bitの符号付き整数|torch.int16 ／ torch.short|int／np.uint16
32-bitの符号付き整数|torch.int32 ／ torch.int|int／np.uint32
64-bitの符号付き整数|torch.int64 ／ torch.long|int／np.uint64
16-bitの浮動小数点|torch.float16 ／ torch.half|float／np.float16
32-bitの浮動小数点|torch.float32 ／ torch.float|float／np.float32
64-bitの浮動小数点|torch.float64 ／ torch.double|float／np.float64


### リスト4-1　チートシート「テンソルの新規作成とサイズ取得／変換」

以下では、シンプルにテンソルの使い方だけを示すためため、`print(x)`などの出力に関するコードは極力、省略した。コードを実行して出力を確認したい場合は、例えば`x = torch.empty(2, 3)`というコードの後に`print(x)`を追記してほしい。また、`x.size()`というコードは、`print(x.size())`のように`print`関数を適宜、自分で補ってほしい。

In [2]:
import torch
import numpy as np

# テンソルの新規作成
x = torch.empty(2, 3) # 2行×3列のテンソル（未初期化状態）を生成
x = torch.rand(2, 3)  # 2行×3列のテンソル（ランダムに初期化）を生成
x = torch.zeros(2, 3, dtype=torch.float) # 2行×3列のテンソル（0で初期化、torch.float型）を生成
x = torch.ones(2, 3, dtype=torch.float)  # 2行×3列のテンソル（1で初期化、torch.float型）を生成
x = torch.tensor([[0.0, 0.1, 0.2],
                  [1.0, 1.1, 1.2]])      # 1行×2列のテンソルをPythonリスト値から作成

# 既存のテンソルを使った新規作成
# 「new_*()」パターン
y = x.new_ones(2, 3)   # 2行×3列のテンソル（1で初期化、既存のテンソルと「同じデータ型」）を生成
# 「*_like()」パターン # 既存のテンソルと「同じサイズ」のテンソル（1で初期化、torch.int型）を生成
y = torch.ones_like(x, dtype=torch.int) 

### リスト4-2　チートシート「テンソルのサイズ取得とサイズ変更」

In [3]:
# テンソルサイズの取得
x.size()               # 「torch.Size([2, 3])」のように、2行3列と出力される
x.shape                # NumPy風の記述も可能。出力は上と同じ
len(x)   # 行数（＝データ数）を取得する際も、NumPy風に記述することが可能
x.ndim   # テンソルの次元数を取得する際も、NumPy風に記述が可能

# テンソルのサイズ変更／形状変更
z = x.view(3, 2)       # 3行2列に変更

### リスト4-3　チートシート「テンソルの演算／計算」

In [4]:
# テンソルの計算操作
x + y                  # 演算子を使う方法
torch.add(x, y)        # 関数を使う方法
torch.add(x, y, out=x) # outパラメーターで出力先の変数を指定可能
x.add_(y)              # 「*_()」パターン。xを置き換えて出力する例（上記のコードと同じ処理）
# PyTorchでは、メソッド名の最後にアンダースコア（_）がある場合（例えば「add_()」）、「テンソルの内部置き換え（in-place changes）が起こること」を意味する。
# アンダースコア（_）がない通常の計算の場合（例えば「add()」）は、計算元のテンソル内部は変更されずに、戻り値として新たなテンソルが取得できる。

tensor([[2.0000, 2.1000, 2.2000],
        [3.0000, 3.1000, 3.2000]])

### リスト4-4　チートシート「テンソルのインデクシング／スライシング」

In [5]:
# インデクシングやスライシング（NumPyのような添え字を使用可能）
print(x)         # 元は、2行3列のテンソル
x[0, 1]          # 1行2列目（※0スタート）を取得
x[:2, 1:]        # 先頭～2行（＝0行目と1行目）×2列～末尾（＝2列目と3列目）の2行2列が抽出される

tensor([[2.0000, 2.1000, 2.2000],
        [3.0000, 3.1000, 3.2000]])


tensor([[2.1000, 2.2000],
        [3.1000, 3.2000]])

### リスト4-5　チートシート「テンソルからPython数値への変換」

In [6]:
# テンソルの1つの要素値を、Pythonの数値に変換
x[0, 1].item()   # 1行2列目（※0スタート）の要素値をPythonの数値で取得

2.0999999046325684

### リスト4-6　チートシート「PyTorchテンソル ←→ NumPy多次元配列値、の変換＆連携」

In [7]:
# PyTorchテンソルを、NumPy多次元配列に変換
b = x.numpy()    # 「numpy()」を呼び出すだけ。以下は注意点（メモリ位置の共有）

# ※PyTorchテンソル側の値を変えると、NumPy多次元配列値「b」も変化する（トラックされる）
print ('PyTorch計算→NumPy反映：')
print(b); x.add_(y); print(b)           # PyTorch側の計算はNumPy側に反映
print ('NumPy計算→PyTorch反映：')
print(x); np.add(b, b, out=b); print(x) # NumPy側の計算はPyTorch側に反映

# -----------------------------------------
# NumPy多次元配列を、PyTorchテンソルに変換
c = np.ones((2, 3), dtype=np.float64) # 2行3列の多次元配列値（1で初期化）を生成
d = torch.from_numpy(c)  # 「torch.from_numpy()」を呼び出すだけ

# ※NumPy多次元配列値を変えると、PyTorchテンソル「d」も変化する（トラックされる）
print ('NumPy計算→PyTorch反映：')
print(d); np.add(c, c, out=c); print(d)  # NumPy側の計算はPyTorch側に反映
print ('PyTorch計算→NumPy反映：')
print(c); d.add_(y); print(c)            # PyTorch側の計算はNumPy側に反映

PyTorch計算→NumPy反映：
[[2.  2.1 2.2]
 [3.  3.1 3.2]]
[[3.  3.1 3.2]
 [4.  4.1 4.2]]
NumPy計算→PyTorch反映：
tensor([[3.0000, 3.1000, 3.2000],
        [4.0000, 4.1000, 4.2000]])
tensor([[6.0000, 6.2000, 6.4000],
        [8.0000, 8.2000, 8.4000]])
NumPy計算→PyTorch反映：
tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[2., 2., 2.],
        [2., 2., 2.]], dtype=torch.float64)
PyTorch計算→NumPy反映：
[[2. 2. 2.]
 [2. 2. 2.]]
[[3. 3. 3.]
 [3. 3. 3.]]


### リスト4-7　チートシート「テンソルのデータ型の変換」

In [8]:
# データ型の変換（※変換後のテンソルには、NumPyの計算は反映されない）
e = d.float()  # 「torch.float64」から「torch.float32」

### リスト4-8　チートシート「テンソル演算でのGPU利用」

ColabでGPUを有効にするには、メニューバーの［ランタイム］－［ランタイムのタイプを変更］をクリックして切り替えてほしい。

In [9]:
# NVIDIAのGPUである「CUDA」（GPU）デバイス環境が利用可能な場合、
# GPUを使ってテンソルの計算を行うこともできる
if torch.cuda.is_available():              # CUDA（GPU）が利用可能な場合
    print('CUDA（GPU）が利用できる環境')
    print(f'CUDAデバイス数： {torch.cuda.device_count()}')
    print(f'現在のCUDAデバイス番号： {torch.cuda.current_device()}')  # ※0スタート
    print(f'1番目のCUDAデバイス名： {torch.cuda.get_device_name(0)}') # 例「Tesla T4」   

    device = torch.device("cuda")          # デフォルトのCUDAデバイスオブジェクトを取得
    device0 = torch.device("cuda:0")       # 1番目（※0スタート）のCUDAデバイスを取得

    # テンソル計算でのGPUの使い方は主に3つ：
    g = torch.ones(2, 3, device=device)    # （1）テンソル生成時のパラメーター指定
    g = x.to(device)                       # （2）既存テンソルのデバイス変更
    g = x.cuda(device)                     # （3）既存テンソルの「CUDA（GPU）」利用
    f = x.cpu()                            # （3'）既存テンソルの「CPU」利用

    # ※（2）の使い方で、GPUは「.to("cuda")」、CPUは「.to("cpu")」と書いてもよい
    g = x.to("cuda")
    f = x.to("cpu")

    # ※（3）の引数は省略することも可能
    g = x.cuda()

    # 「torch.nn.Module」オブジェクト（model）全体でのGPU／CPUの切り替え
    model.cuda()  # モデルの全パラメーターとバッファーを「CUDA（GPU）」に移行する
    model.cpu()   # モデルの全パラメーターとバッファーを「CPU」に移行する
else:
    print('CUDA（GPU）が利用できない環境')

CUDA（GPU）が利用できる環境
CUDAデバイス数： 1
現在のCUDAデバイス番号： 0
1番目のCUDAデバイス名： Tesla K80
